In [ ]:
import os
import sys
import random
import cv2
import math
import time
import numpy as np
import re
import glob
import pandas as pd

ROOT_DIR = os.path.abspath('../models/Mask_RCNN')
DATA_DIR = '../data'
#import mask rcnn
sys.path.append(ROOT_DIR)

from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn.model import log

MODEL_DIR = os.path.join(ROOT_DIR, "logs")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

In [ ]:
from sklearn import model_selection

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from mrcnn import visualize
import scipy.misc

%matplotlib inline 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
class SegmentationConfig(Config):
  NAME = "SegmentationConfig"
  IMAGES_PER_GPU = 2
  NUM_CLASSES = 1 + 9 # background + 9
  STEPS_PER_EPOCH = 1000 #1000

class SegmentationDataset(utils.Dataset):
  ALL_CLASSES = [
              ('bicycle', 35),
              ('bicycle_group', 163),
              ('billboard', 86),
              ('bridge', 98),
              ('building', 97),
              ('bus', 39),
              ('bus_group', 167),
              ('car', 33),
              ('car_groups', 161),
              ('dustbin', 85),
              ('fence', 67),
              ('motorbicycle', 34),
              ('motorbicycle_group', 162),
              ('others', 0),
              ('overpass', 100),
              ('person', 36),
              ('person_group', 164),
              ('pole', 82),
              ('rider', 37),
              ('rider_group', 165),
              ('road', 49),
              ('road_pile', 66),
              ('rover', 1),
              ('siderwalk', 50),
              ('sky', 17),
              ('traffic_cone', 65),
              ('traffic_light', 81),
              ('traffic_sign', 83),
              ('tricycle', 40),
              ('tricycle_group', 168),
              ('truck', 38),
              ('truck_group', 166),
              ('tunnel', 99),
              ('vegatation', 113),
              ('wall', 84)
            ]

  MAIN_CLASSES = [33, 35, 39, 40, 36, 65, 34, 38, 37]

  CLASSES = [ c for c in ALL_CLASSES if c[1] in [33, 35, 39, 40, 36, 65, 34, 38, 37] ]
  CLASS_NAME_TO_ID = {}
  CLASS_ID_TO_NAME = {}
  CLASS_OLD_ID_TO_ID = {}
  for i in range( len(CLASSES) ):
      c_name = CLASSES[i][0]
      old_id = CLASSES[i][1]
      new_id = i + 1

      CLASS_NAME_TO_ID[c_name] = { 'id': new_id,   'old_id': old_id }
      CLASS_ID_TO_NAME[new_id] = { 'name': c_name, 'old_id': old_id }
      CLASS_OLD_ID_TO_ID[old_id] = { 'name': c_name, 'id': new_id }

  def load_cvpr_images(self, images_paths):
    # Add classes
    for class_name in self.CLASS_NAME_TO_ID.keys():
      self.add_class("wad", self.CLASS_NAME_TO_ID[class_name]['id'], class_name)

    for i in range(len(images_paths)):
      path = images_paths[i]
      image_name = path.split('/')[-1].split('.')[0]
      self.add_image("wad", image_id=i, path=path, image_name=image_name)

  def load_mask(self, image_id):
    image_name = self.image_info[image_id]['image_name']
    path = DATA_DIR+"/train_label/"+ image_name +"_instanceIds.png"
    image = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    ids = []

    all_instances = np.unique(image)
    intances_ids = [ int(x/1000) for x in all_instances ]
    intances_ids = np.array([x for x in intances_ids if x in self.MAIN_CLASSES ])

    masks_shape = (image.shape[0], image.shape[1], intances_ids.shape[0])
    masks = np.zeros(masks_shape, dtype=np.bool)

    i_ = 0
    for i in range(all_instances.shape[0]):
      class_id = int(all_instances[i]/1000)
      if self.CLASS_OLD_ID_TO_ID.get(class_id):
        ids.append(self.CLASS_OLD_ID_TO_ID[class_id]['id'])
        masks[:, :, i_] = (image == all_instances[i])
        i_+=1

    ids = np.array(ids, dtype=np.int32)
    return masks, ids


In [ ]:
train = glob.glob(DATA_DIR+'/train_color/*.jpg')
test = glob.glob(DATA_DIR+'/test/*.jpg')
X_train, X_val, _, _ = model_selection.train_test_split(train, train, test_size=0.3)
config = SegmentationConfig()
config.display()
dataset_train = SegmentationDataset()
dataset_train.load_cvpr_images(X_train)
dataset_train.prepare()

dataset_val = SegmentationDataset()
dataset_val.load_cvpr_images(X_val)
dataset_val.prepare()

testset = SegmentationDataset()
testset.load_cvpr_images(test)
testset.prepare()

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)

In [ ]:
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=15, 
            layers='heads')

### Inference

In [ ]:
class InferenceConfig(SegmentationConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)

In [ ]:
log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

### Predict

In [ ]:
results = model.detect([original_image], verbose=1)
r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

In [ ]:
r['rois']

In [ ]:
plt.imshow(original_image[711:883,763:1020, :])

In [ ]:
plt.imshow(r['masks'][:, :, 1][711:883,763:1020])

In [ ]:
len(testset.image_ids)

### Getting the test data results

In [ ]:
from skimage.io import imsave
from multiprocessing import Pool
pool = Pool(processes=8)


In [ ]:
def process_result(r, image_name, image_id):
    data = []
    for i in range(len(r['scores'])):
        mask_idx = i
        confidence = r['scores'][i]
        label_id = r['class_ids'][i]
        mask = r['masks'][:, :, i]
        rois = r['rois'][i]

        data_point = {}
        old_id = testset.CLASS_ID_TO_NAME[label_id]['old_id']
        mask_name = DATA_DIR+'/test_masks/'+ image_name + '_' + str(mask_idx) +'.jpg'
        imsave(mask_name, mask)

        data_point['ImageId'] = image_name
        data_point['LabelId'] = old_id
        data_point['Confidence'] = confidence
        data_point['PixelCount'] = 0
        data_point['rois'] = rois
        data_point['EncodedPixels'] = mask_name

        data.append(data_point)

    return data

In [ ]:
test_results = []
jobs = []
t3 = time.time()
for test_id in testset.image_ids:
    if (test_id % 50) == 0:
        print(test_id)
    results = model.detect([testset.load_image(test_id)], verbose=0)
    r = results[0]

    image_name = testset.image_info[test_id]['image_name']

    p_result = pool.apply_async(process_result, (r, image_name, test_id) )
    jobs.append(p_result)

for job in jobs:
    p_result = job.get(timeout=20)
    test_results = test_results + p_result
t3_ = time.time()
print("Time for all takes:", t3_ - t3)

In [ ]:
print(len(test_results) )
test_results[-1]

In [ ]:
# df_test_results = pd.DataFrame(test_results)

In [ ]:
df_test_results = pd.read_csv(DATA_DIR+'/test_results.csv')
df_test_results.head()

In [ ]:
columns = ['ImageId','LabelId','PixelCount','Confidence','EncodedPixels']

In [ ]:
df_test_results.to_csv(DATA_DIR+'/submit_test_.csv',index=False, columns=columns)

In [ ]:
a=cv2.imread(DATA_DIR+'/test_masks/4e3c95fe41cce1072ccbb69e8a1a20fb_4.jpg')

In [ ]:
a.sum()

In [ ]:
plt.imshow(a*255)

In [ ]:
r['masks'][:, :, 0].shape

In [ ]:

# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))

In [ ]:
r["rois"]

In [ ]:
r["class_ids"]

In [ ]:
r["scores"]

In [ ]:
r

#### Submision File


In [ ]:
d = df_test_results.ix[200]
d

In [ ]:
mask_path = d['EncodedPixels'].split('/')[-1]
mask_im = cv2.imread(DATA_DIR+'/test_masks/'+mask_path, cv2.IMREAD_UNCHANGED)

In [ ]:
mask_bound = [ int(x) for x in d.rois.replace("[", "").replace("]", "").split() ]
mask = mask_im[mask_bound[0]:mask_bound[2], mask_bound[1]:mask_bound[3]]

In [ ]:
plt.imshow(mask*255)
mask.shape

In [ ]:
def find_pixels(mask):
    endcoded_pixels = []
    total_pixels = 0
    for i in range(mask.shape[0]):
        init = 0
        end = 0
        carry = i*mask.shape[1]
        for j in range(1, mask.shape[1]):
            if mask[i, j] > 0 and mask[i, j - 1] == 0:
                init = j + carry

            if mask[i, j] == 0 and mask[i, j - 1] > 0:
                end = j-init + carry
                endcoded_pixels.append( str(init) + ' ' +  str(end) ) 
                total_pixels += end
                init = 0
                end = 0

    return endcoded_pixels, total_pixels

In [ ]:
encodp, _ = find_pixels(mask)

In [ ]:
"|".join(encodp) + "|"

In [ ]:
df_test_results['PixelCount'][3]
df_test_results.shape

In [ ]:
# for i in range(2):
for i in range(df_test_results.shape[0]):
    d = df_test_results.ix[i]
    mask_path = d['EncodedPixels'].split('/')[-1]
    mask_im = cv2.imread(DATA_DIR+'/test_masks/'+mask_path, cv2.IMREAD_UNCHANGED)
    mask_bound = [ int(x) for x in d.rois.replace("[", "").replace("]", "").split() ]
    mask = mask_im[mask_bound[0]:mask_bound[2], mask_bound[1]:mask_bound[3]]
    encoded_pixels, pixels_count = find_pixels(mask)
    encoded_pixels = "|".join(encoded_pixels) + "|"
    df_test_results['PixelCount'][i] = pixels_count
    df_test_results['EncodedPixels'][i] = encoded_pixels

In [ ]:
df_test_results.head()

In [ ]:
#initialize the csv header
with open('my_csv.csv', 'a') as f:
    f.write('ImageId,LabelId,Confidence,PixelCount,EncodedPixels')
    f.write('\n')
    
#abstract the function
def result_function(i):
    d = df_test_results.iloc[i]
    mask_path = d['EncodedPixels'].split('/')[-1]
    mask_im = cv2.imread(DATA_DIR+'/test_masks/'+mask_path, cv2.IMREAD_UNCHANGED)
    mask_bound = [ int(x) for x in d.rois.replace("[", "").replace("]", "").split() ]
    mask = mask_im[mask_bound[0]:mask_bound[2], mask_bound[1]:mask_bound[3]]
    encoded_pixels, pixels_count = find_pixels(mask)
    encoded_pixels = "|".join(encoded_pixels) + "|"
#     df_test_results['PixelCount'][i] = pixels_count
#     df_test_results['EncodedPixels'][i] = encoded_pixels
#     print(df_test_results.ix[i])
    with open('my_csv.csv', 'a') as f:
        #df_test_results.ix[i].to_csv(f, header=False)
        #pd.DataFrame(df_test_results.iloc[i]).to_csv(f, header=False)
        f.write(str(i) + ',' + str(df_test_results.iloc[i]['ImageId']) + ',' + str(df_test_results.iloc[i]['LabelId']) + ',' + str(df_test_results.iloc[i]['Confidence']) + ',' + str(pixels_count) + ',' + str(encoded_pixels))
        f.write('\n')

In [ ]:
pool = multiprocessing.Pool(8)
zip(pool.map(result_function, range(0, df_test_results.shape[0])))
pool.close()